This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [275]:
# get api key from your .env file 

import os 
from dotenv import load_dotenv 

#BaseDir = os.path.abspath(os.path.dirname('/Users/jennyshevchenko/Documents/dsc_v5/Unit 7 - Data Wrangling/7.2 - Data Collection/7.2.5 - API Mini Project/NASDAQ_API_KEY'))

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')



Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

In [74]:
!ls

api_data_wrangling_mini_project.ipynb


For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [75]:
# First, import the relevant modules
import json
import requests
from datetime import datetime,timedelta
import pprint

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [76]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#metadata = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/metadata.json?api_key=" + API_KEY
#url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&limit=1&collapse=day&api_key="+ API_KEY

url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/?collapse=day&api_key" + API_KEY

#start_date=2017-01-01&end_date=2017-12-31&

r = requests.get(url)
Nasdaq_data = r.json()

#pprint.pprint(Nasdaq_data)
print(type(Nasdaq_data))
#this changes the dict file back into json
tojson_=json.dumps(Nasdaq_data)
#print(tojson_)

#for date in by_date: 
#    print('Date: {}'.format(date['newest_available_date']))


<class 'dict'>


In [77]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


for k in Nasdaq_data.keys():
    print(k + ":", Nasdaq_data[k].keys())
    
#pprint.pprint(Nasdaq_data)


dataset: dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [148]:
column_names = Nasdaq_data['dataset']['column_names']  

print(column_names)


['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [161]:
a = "2017-01-01"
b = "2017-12-31"
start_date = datetime.strptime(a, '%Y-%m-%d').date()
end_date = datetime.strptime(b, '%Y-%m-%d').date()

delta = timedelta(days=1)

data = Nasdaq_data['dataset']['data']
#print(data)

#This is for adding column names to the data
#for col_name, dates in zip(Nasdaq_data['dataset']['column_names'], Nasdaq_data['dataset']['data'][0]): 
    
AFX_X_2017=[]
    
for i,v in enumerate(data):
    new = datetime.strptime(v[0],'%Y-%m-%d').date() 
    if new >= start_date and new <= end_date:
        AFX_X_2017.append(v)
            
AFX_TUPLE = tuple(AFX_X_2017)        
#print(AFX_X_2017)
   
    

In [189]:
observations = []
#Column names with coresponding values 

for ele in AFX_X_2017: 
    dict_ele = {}
    for i,in_ele in enumerate(ele):
        dict_ele[column_names[i]]= in_ele
    observations.append(dict_ele)
    
#print(observations)
      

In [269]:
#Calculate what the highest and lowest opening prices were for the stock in this period
#Find the max and minimum of the opening prices
values_are_null =[]
Open_prices =[]

for i in observations:
    if type(i['Open']) != float: 
        values_are_null.append(i)
    else: 
        Open_prices.append(i)

maximum = 0 
minimum = 1000
for op in Open_prices: 
    if maximum <= op['Open']: 
        maximum = op['Open']
    elif minimum >= op['Open']:
        minimum = op['Open']

print("Highest opening:", maximum)
print('Lowest opening:', minimum)    

Highest opening: 53.11
Lowest opening: 34.0


In [270]:
#What was the largest change in any one day(based on High and Low price)
    #change in one day High - Low  

High_Prices = []
Null_High_Prices = []

for i in observations:
    if type(i['High']) != float:
        Null_High_Prices.append(i)
    else: 
        High_Prices.append(i)

#print(High_Prices)

High_minus_Low = []

for i in observations: 
    change = i['High'] - i['Low']
    High_minus_Low.append(change)
#print(High_minus_Low)

Biggest_Change = max(High_minus_Low)
print('Largest Change in one day based on High and Low price:', Biggest_Change)

Largest Change in one day based on High and Low price: 2.8100000000000023


In [271]:
#What was the largest change between any two days (based on Closing Price) 
    #Closing price change betwen two days - date closing price
Closing_Price = []
    
for i in observations: 
    x = i['Close']
    Closing_Price.append(x)

by_date_closing = []

for i,v in enumerate(Closing_Price):
    for i in range(len(Closing_Price)-1):
        change_clo = Closing_Price[i] - Closing_Price[i+1]
        by_date_closing.append(change_clo)
    
Smallest_Closing_Change = min(by_date_closing)
Largest_Closing_Change = max(by_date_closing)

# print(Smallest_Closing_Change)
# print(Largest_Closing_Change)

print('Largest change between any two days:',Smallest_Closing_Change)

Largest change between any two days: -2.559999999999995


In [272]:
#What was the average daily trading volume during this year? (Mean)
Daily_Trading_Volume = []

for i in observations: 
    Daily_Trading_Volume.append(i['Traded Volume'])

Trading_V_Change = sum(Daily_Trading_Volume) / len(Daily_Trading_Volume)

print("Average daily trading volume:", Trading_V_Change)

Average daily trading volume: 89124.33725490196


In [274]:
#What was the median trading volume during this year (Median)
import statistics 
Median = statistics.median(Daily_Trading_Volume)
print('Median trading volume:', Median)

Median trading volume: 76286.0
